# Integrator

## Euler method

- is a first-order numerical procedure for solving ordinary differential equations (ODEs) with a given initial value
- approximating the integral with the finite sum 
- by the Taylor expansion, the quadratic and higher-order terms are ignored

Approximation 
$$ x(t+\tau)=x(t)+\tau\, v(t)\\
v(t+\tau)=v(t)+\tau\, a(t)=v(t)-\frac{\tau}{m}\,\nabla\phi(x(t)) \\
\text{with}\,\, v= \frac{\partial x}{\partial t}; \,\, \text{and}\,\, a=\frac{F}{m}=-\frac{1}{m}\nabla \phi(x(t))$$

### Goals
- setup of the algorithm 
    - parameters  
    - inputs
- timing
- 1D, 2D and 3D distinction
- vectorization
- testing
    - ZeroDivisionError, NameError and TypeErrorn ValueError, Type,Error, RuntimeError, IOError
    - functionality
    - ??
- visualisation
    - timing
    - stability
    - ??

### setup of the algorithm
- import packages
- define function with its inputs
    - (function): potential gradient 
    - (parameter): mass
    - (functionality): size (number of time steps), time step, initial configuration, initial velocity
- creating positiion and velocity container (vector, array, np. zeros ??)
- initialization by adding the start configuration
- time iteration 
    - euler approximation
    - write in pos and vel container
- returning position and velocety container


In [1]:
#%matplotlib notebook
import numpy as np
#import timeit
#import matplotlib.pyplot as plt
#from matplotlib import animation, rc
#from IPython.display import HTML
#import plotly.plotly as py
#import plotly.graph_objs as go

In [ ]:
# wehmeyer
def eulerW(potential_gradient, size=100, x_init=1.0, v_init=0.0, mass=1.0, time_step=0.005):
    x, v = np.zeros(size), np.zeros(size)
    x[0], v[0] = x_init, v_init
    for i in range(1, size):
        x[i] = x[i - 1] + time_step * v[i - 1]
        v[i] = v[i - 1] - time_step * potential_gradient(x[i - 1]) / mass
    return x, v

def potential_gradient(x):
    return x

#timeit.timeit(eulerW, number= 100) # geht nicht warum?

#### euler algorithm for open system 

In [ ]:
def potential_gradient(p,k=10):
    return k*p

In [ ]:
def eulerJ(potential_gradient, position_init, velocity_init, mass, T, time_step):
    size = int(T/time_step)  # number of time steps
    n = len(position_init)   # number of particles
    dim = position_init.shape[-1]
    m = mass
    #print(n)
    # creating positiion and velocity container
    position_matrix, velocity_matrix, acceleration_matrix = np.zeros((size, n, dim)), np.zeros((size, n, dim)), np.zeros((size, n, dim))
    # initialization by adding the start configuration
    position_matrix[0], velocity_matrix[0], acceleration_matrix[0] = position_init, velocity_init, - 1/m *potential_gradient(position_init)
    # time iteration
    for t in range(1, size):
        # rename # kostet das rechenleistung ? bennenung lieber von anfang anders?
        p = position_matrix[t-1]
        v = velocity_matrix[t-1]
        a = acceleration_matrix[t]
        gp = potential_gradient(p)
        # euler approximation
        p_new = p + time_step*v
        #print(v,gp)
        v_new = v - time_step/m * gp
        a = - potential_gradient(p_new)/m
        # write in pos and vel container
        position_matrix[t], velocity_matrix[t], acceleration_matrix[t] = p_new, v_new, a
    # returning position and velocety container
    return position_matrix, velocity_matrix, acceleration_matrix

#%line_profiler

In [ ]:
p_0 = np.array([[1., 0., 0.], [1., 0., 0.]])
v_0 = np.array([[0., 0., 0.], [0., 0., 0.]])
m = np.array([[1.], [1.]])

res = eulerJ(potential_gradient, p_0, v_0, m, 10, 1/100)
res
#print(a[0][:,0,0])
plt.plot(res[0][:,0, 0], res[1][:,0,0])

#### euler algorithm for closed system, non periodic

In [2]:
def potential_gradient(p,k=10):
    return k*p

def boundary(p, p_max):
    p_r = p_max - p_max/4
    p_l = p_max/4
    V_end_r = 4 * np.abs(p-p_r)**2*np.heaviside(np.abs(p-p_r))
    V_end_l = 4 * np.abs(p-p_l)**2*np.heaviside(np.abs(p-p_l))
    return V_end_r + V_end_l

def potential():
    return potantial_gradient + boundary

In [3]:
def euler(potential, position_init, velocity_init, mass, T, time_step):
    size = int(T/time_step)  # number of time steps
    n = len(position_init)   # number of particles
    dim = position_init.shape[-1]
    m = mass
    #print(n)
    # creating positiion and velocity container
    position_matrix, velocity_matrix, acceleration_matrix = np.zeros((size, n, dim)), np.zeros((size, n, dim)), np.zeros((size, n, dim))
    # initialization by adding the start configuration
    position_matrix[0], velocity_matrix[0], acceleration_matrix[0] = position_init, velocity_init, - 1/m *potential_gradient(position_init)
    # time iteration
    for t in range(1, size):
        # rename # kostet das rechenleistung ? bennenung lieber von anfang anders?
        p = position_matrix[t-1]
        v = velocity_matrix[t-1]
        a = acceleration_matrix[t]
        gp = potential_gradient(p)
        # euler approximation
        p_new = p + time_step*v
        #print(v,gp)
        v_new = v - time_step/m * gp
        a = - potential_gradient(p_new)/m
        # write in pos and vel container
        position_matrix[t], velocity_matrix[t], acceleration_matrix[t] = p_new, v_new, a
    # returning position and velocety container
    return position_matrix, velocity_matrix, acceleration_matrix


In [5]:
p_max = np.array([[2., 3., 2.], [2., 3., 2.]])
p_0 = np.array([[1., 0., 0.], [1., 0., 0.]])
v_0 = np.array([[0., 0., 0.], [0., 0., 0.]])
m = np.array([[1.], [1.]])

res = euler(potential_gradient, p_0, v_0, m, 10, 1/100)
res
#print(a[0][:,0,0])
#plt.plot(res[0][:,0, 0], res[1][:,0,0])

(array([[[1.        , 0.        , 0.        ],
         [1.        , 0.        , 0.        ]],
 
        [[1.        , 0.        , 0.        ],
         [1.        , 0.        , 0.        ]],
 
        [[0.999     , 0.        , 0.        ],
         [0.999     , 0.        , 0.        ]],
 
        ...,
 
        [[1.63737293, 0.        , 0.        ],
         [1.63737293, 0.        , 0.        ]],
 
        [[1.63210043, 0.        , 0.        ],
         [1.63210043, 0.        , 0.        ]],
 
        [[1.62519055, 0.        , 0.        ],
         [1.62519055, 0.        , 0.        ]]]),
 array([[[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],
 
        [[-0.1       ,  0.        ,  0.        ],
         [-0.1       ,  0.        ,  0.        ]],
 
        [[-0.2       ,  0.        ,  0.        ],
         [-0.2       ,  0.        ,  0.        ]],
 
        ...,
 
        [[-0.52725052,  0.        ,  0.        ],
         [-0.52725052,  0.   